In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform
from azureml.train.estimator import Estimator

# エクスペリメントの作成
experiment = Experiment(workspace=workspace, name='hyperparameter-tuning')

# 学習スクリプトが格納されているフォルダへのパス
script_folder = './scripts'

# 推定器の設定
estimator = Estimator(source_directory=script_folder,
                      compute_target=compute_target,
                      entry_script='train.py',
                      conda_dependencies_file='conda_dependencies.yml')

# ハイパードライブの設定
hyperparameter_sampling = RandomParameterSampling({
    '--n_estimators': choice(100, 200, 300),
    '--max_depth': choice(5, 10, 15)
})

hyperdrive_config = HyperDriveConfig(estimator=estimator,
                                    hyperparameter_sampling=hyperparameter_sampling,
                                    primary_metric_name='accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4)

# ハイパードライブ実行
hyperdrive_run = experiment.submit(hyperdrive_config)

# ハイパードライブ実行の結果を待機
hyperdrive_run.wait_for_completion(show_output=True)
